In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
import numpy as np
import os
import glob

train_folder = "/kaggle/input/country-flags/flags/train"
test_folder = "/kaggle/input/dataset/quiz_flags"

train_images, train_labels = [], []
test_images, test_labels = [], []

def load_images_from_folder(folder, images_list, labels_list):
    target_size = (64, 64)
    for country_folder in os.listdir(folder):
        country_path = os.path.join(folder, country_folder)
        if os.path.isdir(country_path):
            for img_path in glob.glob(os.path.join(country_path, "*.png")):
                try:
                    img = Image.open(img_path).convert("RGB").resize(target_size)
                    img_array = np.array(img) / 255.0
                    images_list.append(img_array.flatten())
                    labels_list.append(country_folder)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

load_images_from_folder(train_folder, train_images, train_labels)
load_images_from_folder(test_folder, test_images, test_labels)

X_train, y_train = np.array(train_images), np.array(train_labels)
X_test, y_test = np.array(test_images), np.array(test_labels)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

misclassified_indices = np.where(y_test != y_pred)[0]

print("Misclassified Flags:")
for idx in misclassified_indices:
    print(f"Index: {idx}, True Label: {y_test[idx]}, Predicted Label: {y_pred[idx]}")

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Model Accuracy:", accuracy)
print("Classification Report:\n", report)